# CS486 - Artificial Intelligence
## Lesson 9 - Genetic Algorithms

A genetic algorithm is another stochastic approach to iteratively improve a goal. We'll use the AIMA library to implement a string matching genetic algorithm. 

In [1]:
from helpers import romania
from aima.search import *
from aima.notebook import psource

# we'll use pandas to nicely display and filter data
from pandas import DataFrame

## String Matching

We are going to design a genetic algorithm that starts from a population of random strings that, over many generations, will produce a string that matches some target. First, let's choose a target string to match:

In [11]:
target = "Azure FTW!" # start with around 10 characters

Next, e'll use the AIMA **`init_population`** function to generate our initial population:

```python
init_population(pop_size, gene_pool, ind_size)
```

Population size is a tradeoff between computation and memory. For this problem, `100` works well. **`gene_pool`** contains all the possible values that can be used to generate a member of the population. Our gene pool will be the following range of ASCII characters:

In [12]:
pop_size = 100
ind_size = len(target)
gene_pool = [chr(x) for x in range(32, 123)]

print("Gene pool:", ''.join(gene_pool))

Gene pool:  !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz


Finally, the `ind_size` is the number of genes in each individual. 

In [13]:
population = init_population(pop_size,gene_pool,ind_size)

# we'll use a DataFrame to generate a nice table
DataFrame.from_dict({'Individual': population})

,Individual
0,"[W, T, 6, `, H, $, F, (, @, :]"
1,"[k, 2, W, y, (, 1, ,, P, 0, n]"
2,"[1, ,, J, 5, v, d, O, b, j, o]"
3,"[o, j, 8, b, u, c, <, (, z, W]"
4,"[., r, <, t, Q, *, c, ], , )]"
5,"[., ), &, S, !, A, Y, e, ,, t]"
6,"[B, k, o, j, 5, P, >, 9, >, f]"
7,"[\, A, j, 9, k, 3, o, v, =, j]"
8,"[h, q, ), y, k, l, :, B, q, 8]"
9,"[m, `, ], ), 0, 3, /, N, F, X]"


## Fitness 

Next, we need to define a function that quantifies the fitness of individuals in the population. The more fit an individual, the more likely the will combine genes with other members of the population, i.e. "reproduce". 

The fitness function takes an individual - a word in our case - and returns a number. The larger the number returned by the fitness function, the more fit the individual. 

In [14]:
def fitness(word):
    return 0

In [15]:
def fitness(word):
    score = 0
    for i in range(len(word)):
        if word[i] == target[i]:
            score += 1
    return score

The table below shows us the fitness of individuals in the initial population. These are the individuals that will contribute the most to the next generation. 

In [16]:
df = DataFrame.from_dict({
    'Individual': population, 
    'Fitness': [fitness(i) for i in population]
}).sort_values(by=['Fitness'], ascending=False)

df[df['Fitness'] != 0]

,Individual,Fitness
0,"[W, T, 6, `, H, $, F, (, @, :]",1
41,"[J, ], R, #, n, -, M, ], s, !]",1
13,"[h, [, ?, r, K, g, i, ;, %, N]",1


The **`select`** and **`recombine`** functions in the AIMA library select individuals from the population and recombine their genes.

## Mutation

It's unlikely that our population will converge on the goal strictly through reproduction. An important gene may not be represented inour population at all. New genes are introduced into each generation through *mutation*. Mutations are introduced by adding a random gene to randomly selected individuals in each generation.  

Here is the mutation function used by the AIMA library: 

In [17]:
psource(mutate)

Longer strings take longer to converge, so we'll make mutation a function of string size:

In [18]:
mutation_rate = 1 / ind_size

## Re-populating

The following code will use the fitness function to produce a successive generation. 

In [19]:
next_gen = [mutate(recombine(*select(2, population, fitness)), gene_pool, mutation_rate)
              for i in range(len(population))]

The next generation should be significantly more fit than the previous: 

In [20]:
DataFrame.from_dict({
    'Individual': next_gen, 
    'Fitness': [fitness(i) for i in next_gen]
}).sort_values(by=['Fitness'], ascending=False)

,Individual,Fitness
72,"[h, [, ?, r, K, g, i, ;, %, !]",2
41,"[h, [, ?, r, H, $, F, (, @, :]",2
63,"[h, [, ?, r, K, g, F, (, @, :]",2
11,"[h, [, ?, r, K, g, m, ;, s, !]",2
56,"[W, T, 6, `, H, $, F, (, s, !]",2
91,"[h, [, ?, r, n, -, M, ], s, !]",2
16,"[W, T, 6, `, H, $, F, (, s, !]",2
48,"[h, [, ?, r, n, -, M, ], s, !]",2
51,"[h, [, ?, r, n, -, M, ], s, !]",2
45,"[h, [, ?, r, H, $, F, (, @, :]",2


Let's iterate and watch the algorithm progress:

In [22]:
import time
from IPython.core.display import HTML
from IPython.display import clear_output, display

gen_num = 0
next_gen = population[:]
fittest = max(next_gen,key=fitness)

while (fitness(fittest) != ind_size):
    next_gen = [mutate(recombine(*select(2, next_gen, fitness)), gene_pool, mutation_rate)
        for i in range(len(population))]

    gen_num += 1
    fittest = max(next_gen,key=fitness)
    
    if gen_num % 10 == 0:
        display(HTML("<h1>{}: {}</h1>".format(gen_num, ''.join(fittest))))
        clear_output(wait=True)
        
display(HTML("<h1>{}: {}</h1>".format(gen_num, ''.join(fittest))))

We can do all of that in one step using AIMA's **`genetic_algorithm`**:

In [ ]:
%%time
genetic_algorithm(population,fitness,gene_pool,ind_size)

# Writing a Blokus Agent

How can you write an agent using genetic algorithms?